In [2]:
import re
import numpy as np
import pandas as pd
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ShashNavad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ShashNavad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ShashNavad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ShashNavad\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
metadata_path = 'D:/Documents/Project-Long covid detection/C-19_metadata.csv/metadata.csv'
meta_df = pd.read_csv(metadata_path, dtype={'doi': str})
meta_df = meta_df[meta_df.publish_time>"2020-01-01"][['cord_uid','doi','title','abstract', 'publish_time', 'authors', 'journal', 'pdf_json_files']].reset_index(drop=True)
meta_df.rename(columns={'cord_uid':'paper_id'}, inplace=True)
meta_df.head(3)

C:\Users\ShashNavad\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,paper_id,doi,title,abstract,publish_time,authors,journal,pdf_json_files
0,f9tg6xsg,10.1186/s40560-019-0415-z,Dexmedetomidine improved renal function in pat...,BACKGROUND: Dexmedetomidine has been reported ...,2020-01-02,"Nakashima, Tsuyoshi; Miyamoto, Kyohei; Shima, ...",J Intensive Care,document_parses/pdf_json/44449ad1cca160ce491d7...
1,f73c639r,10.1186/s40635-019-0284-8,Aortic volume determines global end-diastolic ...,BACKGROUND: Global end-diastolic volume (GEDV)...,2020-01-02,"Akohov, Aleksej; Barner, Christoph; Grimmer, S...",Intensive Care Med Exp,document_parses/pdf_json/def41c08c3cb1b3752bcf...
2,1qgpa45q,10.1186/s12864-019-6400-z,Whole genome sequencing and phylogenetic analy...,BACKGROUND: Human metapneumovirus (HMPV) is an...,2020-01-02,"Kamau, Everlyn; Oketch, John W.; de Laurent, Z...",BMC Genomics,document_parses/pdf_json/f5ae3f66face323615df3...


In [4]:
print('{} Total papers in original df'.format(meta_df.shape[0]))
print('{} Papers with abstract'.format(len(meta_df[meta_df['abstract'].notnull()])))

606864 Total papers in original df
473781 Papers with abstract


In [5]:
def minimal_clean_text(text):
    text = text.lower()
    text = text.replace('-', ' ')
    text = text.replace('–', ' ')
    return text

def preproc_text(text):
    lemmatizer = WordNetLemmatizer()
    text = re.sub(r'[^A-Za-z ]+', '', text)
    text = minimal_clean_text(text)
    stop_words = set(stopwords.words('english'))
    stop_words.add('long')
    stop_words.add('covid')
    stop_words.add('infection')
    word_tokens = word_tokenize(text)
    cleaned_text = [lemmatizer.lemmatize(token) for token in word_tokens if token not in stop_words and len(token)>1]
    return cleaned_text

In [6]:
# Initial processing

meta_df = meta_df.dropna(subset=['abstract'])
meta_df['abstract'] = meta_df['abstract'].apply(minimal_clean_text)
meta_df.head(3)

,paper_id,doi,title,abstract,publish_time,authors,journal,pdf_json_files
0,f9tg6xsg,10.1186/s40560-019-0415-z,Dexmedetomidine improved renal function in pat...,background: dexmedetomidine has been reported ...,2020-01-02,"Nakashima, Tsuyoshi; Miyamoto, Kyohei; Shima, ...",J Intensive Care,document_parses/pdf_json/44449ad1cca160ce491d7...
1,f73c639r,10.1186/s40635-019-0284-8,Aortic volume determines global end-diastolic ...,background: global end diastolic volume (gedv)...,2020-01-02,"Akohov, Aleksej; Barner, Christoph; Grimmer, S...",Intensive Care Med Exp,document_parses/pdf_json/def41c08c3cb1b3752bcf...
2,1qgpa45q,10.1186/s12864-019-6400-z,Whole genome sequencing and phylogenetic analy...,background: human metapneumovirus (hmpv) is an...,2020-01-02,"Kamau, Everlyn; Oketch, John W.; de Laurent, Z...",BMC Genomics,document_parses/pdf_json/f5ae3f66face323615df3...


In [7]:
# Extract only those abstracts which contain long covid
keywords = ['long covid',
            'post covid 19 syndrome',
            'post covid 19 condition',
            'post acute sequelae of covid 19',
            'chronic covid syndrome']

for keyword in keywords:
    meta_df[keyword.replace(' ', '_')] = meta_df['abstract'].apply(lambda x: True if keyword in x else False)
long_covid_query = '==True or '.join(keyword.replace(' ', '_') for keyword in keywords) + '==True'
long_covid_df = meta_df.query(long_covid_query).reset_index(drop=True)
long_covid_df['tokenized_abstract'] = long_covid_df['abstract'].apply(preproc_text)

print('{:.2f}% of the abstracts contain the long covid related phrases'.format(long_covid_df.shape[0]/meta_df.shape[0]*100))
print('{} papers in total'.format(long_covid_df.shape[0]))

0.24% of the abstracts contain the long covid related phrases
1130 papers in total


In [8]:
long_covid_df['tokenized_abstract'].head()

0    [background, loss, smell, andor, taste, cardin...
1    [people, need, support, physical, emotional, m...
2    [medical, complication, associated, post, sars...
3    [sarscov, virus, lead, variable, clinical, out...
4    [etwa, zehn, prozent, aller, patientinnen, lei...
Name: tokenized_abstract, dtype: object

In [9]:
list_of_all_words = []
for i in long_covid_df['tokenized_abstract']:
    for word in i:
        list_of_all_words.append(word)
print(list_of_all_words)    

['background', 'loss', 'smell', 'andor', 'taste', 'cardinal', 'symptom', 'persistence', 'symptom', 'affect', 'around', 'one', 'fifth', 'people', 'however', 'data', 'regarding', 'clinical', 'resolution', 'loss', 'smell', 'andor', 'taste', 'lacking', 'study', 'ass', 'smell', 'taste', 'loss', 'resolution', 'week', 'follow', 'aim', 'identify', 'risk', 'factor', 'persistent', 'smell', 'loss', 'describe', 'smell', 'loss', 'feature', 'community', 'cohort', 'london', 'known', 'sars', 'cov', 'iggigm', 'antibody', 'status', 'also', 'compare', 'subjective', 'objective', 'smell', 'assessment', 'subset', 'participant', 'method', 'four', 'hundred', 'sixty', 'seven', 'participant', 'acute', 'loss', 'smell', 'andor', 'taste', 'undergone', 'sars', 'cov', 'iggigm', 'antibody', 'testing', 'week', 'earlier', 'completed', 'follow', 'questionnaire', 'resolution', 'symptom', 'subsample', 'participant', 'completed', 'objective', 'olfactory', 'test', 'result', 'compared', 'subjective', 'smell', 'evaluation', '

In [10]:
# Simple word query
print(f'"male" appears {list_of_all_words.count("male")} time(s)')
print(f'"female" appears {list_of_all_words.count("female")} time(s)')
print(f'"old" appears {list_of_all_words.count("old")} time(s)')
print(f'"youth" appears {list_of_all_words.count("diabetic")} time(s)')

"male" appears 82 time(s)
"female" appears 205 time(s)
"old" appears 90 time(s)
"youth" appears 4 time(s)


In [11]:
df = pd.value_counts(np.array(list_of_all_words))
df.head(20)

patient       2951
symptom       2791
sars          1300
cov           1286
post          1233
study         1202
acute         1199
syndrome      1062
disease        969
health         800
fatigue        790
reported       749
may            667
month          663
result         637
clinical       610
term           574
care           554
associated     548
persistent     533
dtype: int64

In [12]:
# Positional Posting list
line = len(long_covid_df['tokenized_abstract'])
pos_index = {}
fileno = 0
for i in range(line):
    for pos,term in enumerate(long_covid_df['tokenized_abstract'][i]):
        if term in pos_index:
            pos_index[term][0] = pos_index[term][0] + 1
            if fileno in pos_index[term][1]:
                pos_index[term][1][fileno].append(pos)
            else:
                pos_index[term][1][fileno] = [pos]
        else:
                        pos_index[term] = []
                        pos_index[term].append(1)
                        pos_index[term].append({})
                        pos_index[term][1][fileno] = [pos]
    fileno += 1

In [13]:
for key, value in pos_index.items():
    print(key, ' : ', value)

background  :  [237, {0: [0], 10: [0], 11: [0], 12: [0], 14: [0], 15: [0], 17: [0], 24: [0], 27: [0], 28: [0], 31: [0], 55: [0], 60: [0], 61: [0], 66: [0], 67: [0], 78: [0], 81: [35], 88: [0], 89: [0], 96: [0], 102: [0], 114: [0], 122: [0, 91], 123: [0], 128: [0, 12], 130: [0], 131: [0], 132: [0], 133: [0], 136: [0], 143: [0], 144: [81], 159: [0], 162: [0], 166: [0], 167: [0], 172: [0], 175: [0], 176: [0], 177: [0], 178: [0], 180: [0], 181: [0], 182: [0], 183: [0], 191: [0], 192: [0, 92], 194: [0], 205: [0], 209: [0], 210: [0], 222: [7], 223: [7], 225: [0], 226: [0], 250: [0], 252: [0], 253: [0], 259: [0], 268: [0], 279: [0], 280: [0], 285: [0], 286: [0], 291: [0], 302: [0], 303: [0], 304: [0, 93], 334: [0], 349: [0], 350: [0], 355: [0], 363: [0], 364: [0], 368: [0], 375: [0], 376: [0], 379: [0], 380: [0], 383: [0], 391: [0], 418: [0], 419: [0], 426: [97], 439: [0], 440: [0], 481: [0], 483: [0], 493: [0], 494: [0], 500: [0], 501: [0], 514: [0], 525: [92], 538: [0], 543: [0], 548: [0], 

localized  :  [6, {3: [17], 237: [52], 238: [52], 289: [18], 290: [18], 622: [52]}]
respiratory  :  [443, {3: [18], 5: [49], 7: [12, 60], 8: [72, 93], 10: [10], 11: [8], 20: [9], 21: [84], 23: [44, 60, 73], 24: [55], 26: [12], 28: [13, 42, 127], 29: [19, 28], 32: [3], 33: [33], 40: [4], 43: [42], 46: [21], 50: [2], 60: [65], 70: [12], 72: [50], 74: [43], 79: [6, 12], 80: [3, 26, 74, 113, 130, 136], 83: [5, 12], 89: [55, 103, 134], 96: [5], 112: [4], 124: [51], 128: [72, 108, 161], 130: [7], 138: [2], 159: [12, 129], 162: [6], 163: [8], 174: [2], 183: [101], 186: [161], 187: [133, 196], 191: [57], 194: [4], 198: [9], 203: [7], 207: [2], 208: [2], 211: [11], 212: [11], 229: [84], 230: [84], 237: [30, 35, 83], 238: [30, 35, 83], 242: [26, 132, 149], 245: [9, 20], 246: [50], 255: [7], 260: [2, 25, 72, 111, 128, 133, 226, 259, 269, 288], 264: [64, 81], 265: [64, 81], 266: [12], 267: [12], 268: [98], 274: [10], 276: [32, 78], 277: [32, 78], 278: [5, 12], 285: [115], 286: [115], 288: [18, 27]

novel  :  [65, {6: [40], 59: [84], 81: [2], 94: [112], 95: [45], 103: [61], 120: [5], 134: [3], 156: [110], 174: [121], 187: [35, 171], 203: [58], 264: [2], 265: [2], 364: [87], 372: [113], 388: [2], 389: [2], 390: [2], 411: [110], 412: [110], 416: [8, 74], 417: [8, 74], 437: [57], 457: [5], 471: [412], 505: [13], 506: [13], 534: [0], 535: [0], 536: [0], 562: [2], 590: [34, 77], 591: [45], 593: [1], 618: [0], 678: [5], 691: [13], 708: [113], 709: [2], 712: [63], 733: [16], 749: [189], 765: [2], 779: [8], 797: [110], 883: [112], 892: [4], 912: [17], 937: [0], 942: [3], 960: [2], 985: [2], 991: [30], 998: [25], 1048: [5], 1062: [2], 1087: [18], 1106: [5], 1109: [8, 74]}]
monitored  :  [9, {6: [45], 42: [146], 50: [125], 138: [47], 572: [29], 675: [109], 706: [26], 860: [139], 988: [181]}]
change  :  [189, {6: [46], 22: [59], 23: [6], 50: [35, 93, 133, 142, 154, 166], 55: [121], 59: [73], 70: [121], 83: [50], 88: [113], 93: [36, 68], 97: [165], 99: [3, 80], 102: [162], 122: [94], 137: [38

history  :  [95, {8: [113], 10: [45], 24: [21, 31], 26: [36], 40: [52], 47: [26], 61: [63, 65], 97: [101, 130, 154], 99: [30], 106: [31], 111: [48], 134: [60, 198], 144: [181], 186: [33], 193: [94], 213: [63, 65], 233: [34], 234: [34], 239: [47], 252: [64], 253: [64], 255: [36], 256: [25], 266: [36], 267: [36], 285: [86], 286: [86], 294: [39], 295: [39], 310: [31], 370: [91], 386: [38], 387: [38], 401: [26], 402: [26], 422: [122], 423: [122], 426: [181], 431: [203], 489: [27], 515: [32], 526: [45, 55], 543: [56, 90], 549: [53], 561: [64], 564: [63], 568: [86], 590: [83], 604: [111], 621: [66], 653: [170], 655: [41], 668: [130], 672: [75], 679: [205], 687: [45, 55], 702: [21, 36], 727: [64], 743: [4], 746: [103], 751: [24], 769: [109], 770: [21, 27], 786: [44], 808: [100], 834: [38], 865: [32], 953: [34], 955: [29], 956: [62], 958: [31], 959: [48], 976: [57], 986: [27], 998: [63], 1005: [49], 1006: [52], 1012: [45], 1020: [36], 1053: [134], 1070: [100], 1078: [91], 1093: [44], 1107: [34

education  :  [57, {12: [232, 261], 21: [97], 118: [40, 90, 119, 123], 125: [36, 138], 162: [58], 165: [102], 190: [112], 222: [15, 22, 131, 145], 223: [15, 22, 131, 145], 229: [97], 230: [97], 359: [104], 363: [226, 255], 391: [27, 58, 67, 157], 394: [123], 395: [123], 427: [118], 462: [102], 507: [42], 520: [112], 531: [83], 532: [83], 555: [8, 144], 636: [83], 729: [62], 749: [47, 54], 821: [123], 867: [118], 909: [29, 72, 77], 913: [152], 940: [27, 58, 67, 157], 993: [49, 84, 122], 1029: [242]}]
grant  :  [21, {12: [233, 262, 278], 75: [78], 128: [216], 291: [183], 335: [172, 187], 355: [219, 226], 363: [227, 256, 271], 456: [222], 833: [157], 911: [183], 968: [197, 210], 1014: [224], 1080: [222, 229]}]
supported  :  [37, {12: [234, 263], 75: [81], 88: [139], 136: [58], 335: [154], 352: [25], 363: [228, 257], 408: [265], 416: [58], 417: [58], 456: [170, 191, 195, 199, 203, 207, 211], 534: [61], 535: [61], 536: [61], 550: [148], 636: [123, 162, 178, 192, 201], 783: [142], 833: [156,

...

enquired  :  [2, {756: [266], 786: [61]}]
merit  :  [1, {756: [274]}]
noting  :  [1, {756: [275]}]
speculative  :  [1, {756: [315]}]
madness  :  [1, {756: [317]}]
purported  :  [1, {756: [321]}]
incurability  :  [1, {756: [322]}]
discernible  :  [1, {756: [377]}]
talking  :  [1, {756: [382]}]
stranger  :  [1, {756: [383]}]
deterrent  :  [1, {756: [405]}]
substantiating  :  [1, {756: [426]}]
notion  :  [1, {756: [427]}]
blamed  :  [1, {756: [451]}]
overarching  :  [2, {756: [461], 970: [57]}]
diseasesconditions  :  [1, {756: [466]}]
disadvantage  :  [1, {756: [472]}]
prejudice  :  [1, {756: [500]}]
favouring  :  [1, {756: [523]}]
adaptability  :  [1, {756: [524]}]
psp  :  [4, {757: [24, 35, 44, 67]}]
dendritic  :  [2, {757: [52], 1056: [21]}]
compartment  :  [3, {757: [66], 906: [70], 1044: [37]}]
augmentation  :  [2, {757: [75], 894: [45]}]
reverted  :  [1, {757: [81]}]
amelioration  :  [4, {757: [90], 862: [61, 131], 1121: [120]}]
immunopathogenic  :  [1, {757: [94]}]
nonprobability  

apontando  :  [1, {812: [193]}]
hemlise  :  [1, {812: [197]}]
possibilidade  :  [1, {812: [200]}]
evans  :  [4, {812: [204, 255, 266, 307]}]
iniciada  :  [2, {812: [206, 256]}]
infuso  :  [1, {812: [207]}]
imunoglobulina  :  [1, {812: [209]}]
resposta  :  [1, {812: [212]}]
satisfatria  :  [1, {812: [213]}]
intervenes  :  [1, {812: [214]}]
evoluiu  :  [1, {812: [216]}]
bito  :  [1, {812: [218]}]
duas  :  [2, {812: [219, 275]}]
semanas  :  [1, {812: [220]}]
incio  :  [1, {812: [222]}]
hematolgico  :  [1, {812: [224]}]
discusso  :  [1, {812: [225]}]
uma  :  [4, {812: [226, 250, 267, 308]}]
recente  :  [1, {812: [228]}]
manifestaes  :  [2, {812: [231, 242]}]
laboratoriais  :  [1, {812: [233]}]
descritas  :  [1, {812: [234]}]
cada  :  [1, {812: [235]}]
dia  :  [1, {812: [236]}]
termo  :  [1, {812: [237]}]
covidfoi  :  [1, {812: [238]}]
criado  :  [1, {812: [239]}]
representar  :  [1, {812: [241]}]
tardias  :  [1, {812: [243]}]
esse  :  [2, {812: [245, 322]}]
descreve  :  [1, {812: [249]}]
m

dopamine  :  [1, {891: [45]}]
enterocytes  :  [3, {891: [59, 97, 130]}]
seq  :  [1, {891: [84]}]
dopaminetrace  :  [1, {891: [107]}]
serving  :  [1, {891: [121]}]
unknownmost  :  [1, {892: [31]}]
today  :  [1, {892: [45]}]
availablein  :  [1, {892: [62]}]
predominant  :  [3, {894: [5], 924: [94], 944: [213]}]
dysregulations  :  [1, {894: [14]}]
cribriform  :  [1, {894: [50]}]
plate  :  [1, {894: [51]}]
congestion  :  [1, {894: [53]}]
travel  :  [4, {894: [73], 981: [112], 997: [4, 155]}]
cascade  :  [3, {894: [83, 139], 971: [92]}]
excitotoxicity  :  [1, {894: [103]}]
tmprss  :  [2, {894: [112], 1104: [65]}]
destruction  :  [2, {894: [129], 1115: [135]}]
symptomssigns  :  [1, {896: [68]}]
blunting  :  [1, {896: [85]}]
conglomeration  :  [1, {896: [87]}]
syrian  :  [1, {897: [8]}]
gsh  :  [6, {897: [10, 27, 47, 62, 92, 97]}]
resembles  :  [1, {897: [14]}]
undefined  :  [1, {897: [26]}]
silico  :  [1, {897: [29]}]
remarkably  :  [1, {897: [57]}]
cvc  :  [2, {897: [66, 108]}]
thickening  

ssri  :  [1, {1001: [75]}]
serotonin  :  [1, {1001: [77]}]
reuptake  :  [1, {1001: [78]}]
antidepressant  :  [1, {1001: [81]}]
fluoxetine  :  [1, {1001: [82]}]
broadening  :  [1, {1001: [88]}]
freiburg  :  [1, {1003: [30]}]
departmentsspecialists  :  [1, {1003: [69]}]
yielding  :  [2, {1003: [104], 1045: [122]}]
empirically  :  [1, {1005: [32]}]
anhedonia  :  [2, {1005: [79, 95]}]
alerting  :  [2, {1005: [88, 110]}]
orienting  :  [2, {1005: [89, 111]}]
intact  :  [1, {1005: [114]}]
implicate  :  [1, {1005: [148]}]
arabic  :  [1, {1006: [42]}]
manifested  :  [1, {1006: [116]}]
palpation  :  [1, {1006: [130]}]
coordinated  :  [3, {1007: [61], 1010: [50], 1049: [75]}]
reconstitute  :  [1, {1007: [80]}]
nkt  :  [1, {1008: [41]}]
igacdcd  :  [1, {1008: [48]}]
hyperactivated  :  [1, {1008: [52]}]
hladrcd  :  [1, {1008: [53]}]
hepatocyte  :  [1, {1008: [61]}]
quasi  :  [1, {1008: [81]}]
tfts  :  [3, {1009: [44, 93, 111]}]
reassessed  :  [1, {1009: [65]}]
tpo  :  [4, {1009: [104, 124, 142, 150

rosmarinic  :  [1, {1108: [36]}]
nlrp  :  [4, {1108: [55, 75, 123, 134]}]
inflammasome  :  [3, {1108: [56, 124, 135]}]
exerts  :  [1, {1108: [58]}]
neuroprotective  :  [2, {1108: [62, 159]}]
keapnrf  :  [1, {1108: [70]}]
attenuates  :  [1, {1108: [74]}]
emanating  :  [1, {1108: [96]}]
unregulated  :  [2, {1108: [99, 122]}]
endovascular  :  [1, {1108: [104]}]
efficacious  :  [1, {1108: [139]}]
systemically  :  [1, {1108: [149]}]
penetrate  :  [1, {1108: [151]}]
parenchyma  :  [1, {1108: [157]}]
countermeasure  :  [1, {1108: [171]}]
syndromal  :  [1, {1110: [48]}]
fogginess  :  [1, {1110: [52]}]
perpetuated  :  [1, {1110: [68]}]
neuroimmunological  :  [1, {1110: [76]}]
allergen  :  [1, {1110: [99]}]
pollutant  :  [1, {1110: [100]}]
reliance  :  [1, {1111: [51]}]
nonresponse  :  [1, {1111: [59]}]
tunisia  :  [1, {1112: [57]}]
histologically  :  [1, {1113: [72]}]
organizing  :  [1, {1113: [74]}]
tlc  :  [1, {1113: [102]}]
radiomorphological  :  [1, {1113: [110]}]
electrocardiographic  :  [

In [52]:
def get_positions(posting_values, doc):
    for key, value in posting_values.items():
        if key == doc:
            return value
    return []

In [44]:
def gen_init_set_matchings(word):
    init = []
    word_postings = pos_index[word][1]
    for key, value in word_postings.items():
        for val in value:
            init.append((key, val))
    return init

In [54]:
def match_positional_index(init, b):
    matched_docs = []
    for p in init:
        doc = p[0]
        pos = p[1]
        count = 0

        for k in b:
            pos = pos+1
            k_pos = pos_index[k][1]
            docs_list = [key for key, value in k_pos.items()]
            if doc in docs_list:
                doc_positions = get_positions(k_pos, doc)
                if pos in doc_positions:
                    count += 1
                else:
                    count += 1
                    break

            if count == len(b):
                matched_docs.append(p[0])
    return set(matched_docs)

In [73]:
def run_query(query_tokens):
    
    init_word = query_tokens[0]
    
    if(len(query_tokens) == 1):
        k_pos = pos_index[init_word][1]
        docs_list = [key for key, value in k_pos.items()] 
        return set(docs_list)
    
    init_matches = gen_init_set_matchings(init_word)    
    
    query_tokens.pop(0)
    total_matched_docs = match_positional_index(init_matches, query_tokens)
    return total_matched_docs

In [74]:
query1 = ['coronary', 'heart', 'disease']

In [76]:
lists = run_query(query1)
print(lists)

{769, 685, 529, 530, 562, 594, 1011, 854, 825}


In [81]:
query2 = ['high', 'blood', 'pressure']

In [82]:
lists = run_query(query2)
print(lists)

{465, 466, 1066}


In [89]:
query3 = ['suffering', 'diabetes']

In [91]:
lists = run_query(query3)
print(lists)

{896, 901, 1029, 266, 267, 909, 142, 272, 273, 529, 530, 789, 534, 535, 536, 662, 26, 790, 917, 1049, 551, 424, 553, 679, 937, 556, 685, 560, 816, 311, 312, 59, 699, 958, 832, 705, 202, 1098, 594, 211, 212, 476, 477, 733, 351, 736, 992, 105, 1011, 628, 373, 118, 374, 763, 382}
